In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
torch.manual_seed(42)

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170498071/170498071 [00:12<00:00, 13619862.76it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
n_classes = 10
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, n_classes)
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 94.7MB/s]


In [6]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [18]:
from tqdm import tqdm

def train(n_epochs: int):
    for epoch in range(n_epochs):
        model.train()
        train_loss, train_acc = 0, 0

        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, pred = torch.max(outputs, 1)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            train_acc += torch.sum(pred == labels.data)

        epoch_loss = train_loss / len(train_dataset)
        epoch_acc = (train_acc.double() / len(train_dataset))*100
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.3f}, Acc: {epoch_acc:.3f}%")

In [19]:
def eval():
    model.eval()
    test_loss, test_acc = 0, 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, pred = torch.max(outputs, 1)

            test_loss += loss.item() * inputs.size(0)
            test_acc += torch.sum(pred == labels.data)

    test_loss = test_loss / len(test_dataset)
    test_acc = (test_acc.double() / len(test_dataset))*100
    print(f"Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.3f}%")

In [20]:
num_epochs: int = 10

train(num_epochs)

100%|██████████| 782/782 [01:16<00:00, 10.21it/s]


Epoch 1/10, Loss: 1.664, Acc: 32.200%


100%|██████████| 782/782 [01:15<00:00, 10.37it/s]


Epoch 2/10, Loss: 1.186, Acc: 56.736%


100%|██████████| 782/782 [01:15<00:00, 10.40it/s]


...

Epoch 8/10, Loss: 0.614, Acc: 81.516%


100%|██████████| 782/782 [01:15<00:00, 10.36it/s]


Epoch 9/10, Loss: 0.590, Acc: 82.416%


100%|██████████| 782/782 [01:15<00:00, 10.37it/s]

Epoch 10/10, Loss: 0.566, Acc: 82.992%


In [21]:
eval()

100%|██████████| 157/157 [00:05<00:00, 30.83it/s]

Test Loss: 0.699, Test Acc: 79.620%
